In [6]:
import duckdb
from minio import Minio

# Connect to MinIO
minio_client = Minio(
    "localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

# Create a bucket and upload a sample file (if not exists)
#if not minio_client.bucket_exists("test"):
    #minio_client.make_bucket("test")
    
# Upload a sample CSV file to MinIO (example)
# minio_client.fput_object("test", "data.csv", "local-data.csv")

# Configure DuckDB for S3 access
conn = duckdb.connect()
conn.execute("""
        INSTALL httpfs;
        LOAD httpfs;
        SET s3_endpoint='minio:9000';
        SET s3_access_key_id='minioadmin';
        SET s3_secret_access_key='minioadmin';
        SET s3_use_ssl=false;
        SET s3_url_style='path';
""")

# Query data from MinIO
result = conn.execute("""
    SELECT * 
    from read_parquet(
            's3a://blake/silver/breweries/**/*.parquet',
            hive_partitioning=1
        )
    LIMIT 10
""").fetchdf()

print(result)

                    address_1 address_2 brewery_type  \
0              Unnamed Street  Clonmore        micro   
1        3000 E Ray Rd Bldg 6      None        micro   
2                        None      None        micro   
3  2045 N Forbes Blvd Ste 105      None        micro   
4                        None      None        micro   
5         1304 Scott St Ste D      None       closed   
6                   1501 E St      None        large   
7   31125 Via Colinas Ste 907      None        micro   
8                1043 Park St      None        micro   
9              2620 Walnut St      None        large   

                                     id     latitude     longitude  \
0  e5f3e72a-fee2-4813-82cf-f2e53b439ae6  52.84930763  -6.979343891   
1  fb94830f-6196-4f59-9189-c9060b778085         None          None   
2  0faa0fb2-fffa-416d-9eab-46f67477c8ef    33.436188  -111.5860662   
3  4f4b5b34-d572-4dff-a18f-47e507c073e6  32.24673727  -110.9927505   
4  fe6b9893-b93e-43d5-a9f6-3e0c89